In [ ]:
from connect4.connect_four import ConnectFour
from utils import cls, draw_matrix


game = ConnectFour()
state = game.initial_state

valid_moves = game.game_rows * game.game_cols
won = False
current_player = 0
while not won and valid_moves > 0:
  column = input(f"Enter column for player {current_player}: ")
  cls()
  state, won = game.move(state, int(column), current_player)
  draw_matrix(game.decode_binary(state))
  if won:
    print(f"Player {current_player} won!")
  valid_moves = len(game.possible_moves(state))
  current_player = 1 - current_player

if valid_moves == 0:
  print("It's a draw")


In [ ]:
import numpy as np
import torch
from connect4.connect_four import ConnectFour
from mcts.mcts import MCTS
from model.model_64 import Net_64
from utils import cls, draw_matrix
import config as cfg

game = ConnectFour()
mcts_store = MCTS(game)
state = game.initial_state

model = Net_64(input_shape=game.obs_shape, actions_n=game.action_space)
model.load_state_dict(torch.load(
            'trained/trained_connect4/best_026_12000.dat', map_location=lambda storage, loc: storage))

won = False
valid_moves = game.game_rows * game.game_cols

while not won and valid_moves > 0:
  column = input("Enter column: ")
  cls()
  print("column: ", column)
  state, won = game.move(state, int(column), 0)
  draw_matrix(game.decode_binary(state))
  if won:
    print("You won!")
    break
  valid_moves = len(game.possible_moves(state))
  if valid_moves == 0:
    break

  mcts_store.search_batch(
              cfg.BOT_MCTS_SEARCHES, cfg.BOT_MCTS_BATCH_SIZE, state, 1, model)
  probs, values = mcts_store.get_policy_value(state, tau=0)
  action = np.random.choice(game.action_space, p=probs)
  state, won = game.move(state, action, 1)
  draw_matrix(game.decode_binary(state))
  if won:
    print("You lose!!!")
    break
  valid_moves = len(game.possible_moves(state))

if valid_moves == 0:
  print("It's a draw")
